In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

features_data = pd.read_csv('features_30_sec.csv')
print(features_data.head(5))

          filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.wav  661794          0.350088         0.088757  0.130228   
1  blues.00001.wav  661794          0.340914         0.094980  0.095948   
2  blues.00002.wav  661794          0.363637         0.085275  0.175570   
3  blues.00003.wav  661794          0.404785         0.093999  0.141093   
4  blues.00004.wav  661794          0.308526         0.087841  0.091529   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.002827             1784.165850          129774.064525   
1  0.002373             1530.176679          375850.073649   
2  0.002746             1552.811865          156467.643368   
3  0.006346             1070.106615          184355.942417   
4  0.002303             1835.004266          343399.939274   

   spectral_bandwidth_mean  spectral_bandwidth_var  ...  mfcc16_var  \
0              2002.449060            85882.761315  ...   52.420910   
1              2039.036516        

In [2]:
# count of each genre in the dataset
print(features_data['label'].value_counts())

label
blues        100
classical    100
country      100
disco        100
hiphop       100
jazz         100
metal        100
pop          100
reggae       100
rock         100
Name: count, dtype: int64


In [3]:
# converting the labels into integers using label encoder
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
features_data['label'] = label_encoder.fit_transform(features_data['label'])

KNN Algorithm

In [4]:
from scipy.stats import mode

class KNN_classifier():
  def __init__(self,k):
    self.k = k
  
  def fit(self,x_train,y_train):
    self.x_train = x_train
    self.y_train = y_train
    self.r_train,self.c_train = x_train.shape

  def euclidean_distance(self,x,y):
    d = np.sqrt(np.sum(np.square(x-y)))
    return d
  
  def predict(self,x_test):
    self.x_test = x_test
    self.r_test,self.c_test = x_test.shape
    predictions = np.zeros(self.r_test)
    for i in range(self.r_test):
      x = self.x_test[i]
      neighbors = np.zeros(self.k)
      neighbors = self.k_nearest_neighbours(x)
      predictions[i] = mode(neighbors, keepdims=True)[0][0]
    return predictions

  def k_nearest_neighbours(self,x):
    distances = np.zeros(self.r_train)
    for i in range(self.r_train):
      d = self.euclidean_distance(x,self.x_train[i])
      distances[i] = d
    indices = distances.argsort()
    y_train_res = self.y_train[indices]
    return y_train_res[:self.k]

In [5]:
# dropping the filename column since it is not relevant and also dropping label for splitting
x = features_data.drop(['filename','label'],axis=1)
y = features_data['label']

In [6]:
# scaling the data using MinMax Scaling
# instantiating the minmax scaler
cols = x.columns
minmaxscaler = preprocessing.MinMaxScaler()
scaled_values = minmaxscaler.fit_transform(x)
# creating new data frame with the scaled values
nf = pd.DataFrame(scaled_values, columns=cols)

In [7]:
# splitting the dataset into training and testing set
from sklearn.model_selection import train_test_split
x = nf.values
y = y.values
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size = 0.3,random_state = 33)

KNN Classifier

In [9]:
model = KNN_classifier(k=5)
model.fit(X_train,Y_train)
Y_predictions = model.predict(X_test)
cnt = 0
for i in range(len(Y_predictions)):
  if Y_test[i] == Y_predictions[i]:
    cnt+=1
print(f"Accuracy is {cnt/len(Y_test)*100}")

Accuracy is 64.33333333333333
